In [1]:
%load_ext autoreload 
%autoreload 2
import process as p
import pandas as pd
import dmc

In [2]:
df = p.processed_data()
df = p.shuffle(df)

In [3]:
start, end, split = pd.Timestamp('2014-1-1'), pd.Timestamp('2014-12-31'), pd.Timestamp('2014-10-1')
df.orderDate = pd.to_datetime(df.orderDate)
mask = (df.orderDate >= start) & (df.orderDate <= end)
df_full = df[mask]
te_size = 10000
tr_size = 10000
X, Y, fts = dmc.transformation.transform_preserving_header(df_full, scaler=dmc.transformation.scale_features, binary_target=True)
train = X[:tr_size], Y[:tr_size]
test = X[tr_size:tr_size+te_size], Y[tr_size:tr_size+te_size]

/home/bpm1/BigData/run-dmc/venv/lib/python3.4/site-packages/pandas/core/generic.py:2701: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [4]:
#res_tree = dmc.evaluation.evaluate_features_leaving_one_out(train[0], train[1], test[0], test[1], fts, dmc.classifiers.DecisionTree)

In [5]:
'''Calculate impact of features for n iterations'''
impact_map = dict()
feature_set = set(fts)
for feature in fts:
        impact_map[feature] = 0.
for i in range(10):
    print(i)
    res_tree = dmc.evaluation.evaluate_features_leaving_one_out(train[0], train[1], test[0], test[1], fts, dmc.classifiers.DecisionTree)
    for feature in feature_set:
            impact_map[feature] = impact_map[feature] + res_tree.decrement[feature]
import operator
sorted_impact_list = sorted(impact_map.items(), key=operator.itemgetter(1))
print(sorted_impact_list)


0
1
2
3
4
5
6
7
8
9
[('customerAvgUnisize', -0.031600000000000183), ('t_order_meanPrice', -0.027100000000000124), ('binnedColorCode', -0.026700000000000279), ('voucherID', -0.026399999999999979), ('orderWeek', -0.021700000000000275), ('t_order_boughtArticleCount', -0.018199999999999883), ('deviceID', -0.016200000000000325), ('t_order_mostExpensivePrice', -0.016000000000000125), ('t_order_sameArticlesCount', -0.015900000000000136), ('t_reducedPaymentMethod_A', -0.014399999999999857), ('t_order_cheapestPrice', -0.013400000000000412), ('productGroup', -0.013200000000000212), ('t_wsv', -0.010900000000000354), ('voucherSavings', -0.010800000000000143), ('t_order_hasVoucher_A', -0.010500000000000287), ('orderWeekDay', -0.009600000000000164), ('relativeSavings', -0.0095000000000000639), ('paymentMethod', -0.0094999999999999529), ('t_customer_voucherCount', -0.009300000000000197), ('t_paymentWithFee_A', -0.00900000000000023), ('t_isWeekend_A', -0.008999999999999897), ('t_isTypeBelt', -0.008900

In [6]:
res_tree[(res_tree.decrement < 0)]

,decrement,precision
t_order_boughtArticleCount,-0.0007,0.6965
t_order_mostExpensivePrice,-0.0043,0.7001
binnedColorCode,-0.0007,0.6965
t_isTypeTop,-0.0009,0.6967
orderDay,-0.0014,0.6972
t_order_sameArticlesCount_DiffSize,-0.0014,0.6972
sizeReturnProb,-0.0012,0.6970
relativeSavings,-0.0033,0.6991
totalSavings,-0.0017,0.6975
t_ssv,-0.0052,0.7010


In [7]:
kick_features = 20
for i in range(kick_features):
    improvement_without_feature = dmc.evaluation.evaluate_without_one_feature(train[0], train[1], test[0], test[1], fts, sorted_impact_list[i][0], dmc.classifiers.DecisionTree, 20)
    print(improvement_without_feature)

Improvement without customerAvgUnisize: 0.0012500000000003064
Improvement without t_order_meanPrice: 0.0019849999999999035
Improvement without binnedColorCode: 0.0028249999999999664
Improvement without voucherID: 0.0034250000000001224
Improvement without orderWeek: 0.0012750000000000261
Improvement without t_order_boughtArticleCount: -0.0005700000000001815
Improvement without deviceID: 0.0008550000000001612
Improvement without t_order_mostExpensivePrice: -4.999999999810711e-06
Improvement without t_order_sameArticlesCount: 0.0016049999999999676
Improvement without t_reducedPaymentMethod_A: 0.0007649999999999046
Improvement without t_order_cheapestPrice: 0.002274999999999916
Improvement without productGroup: 0.0020450000000000745
Improvement without t_wsv: -0.00043499999999985217


KeyboardInterrupt: 

In [11]:
n = 5
impact_map = dict()
feature_set = set(fts)
for feature in fts:
    impact_map[feature] = 0.

for j in range(n):
    print(j)
    df = p.shuffle(df)
    start, end, split = pd.Timestamp('2014-1-1'), pd.Timestamp('2014-12-31'), pd.Timestamp('2014-10-1')
    df.orderDate = pd.to_datetime(df.orderDate)
    mask = (df.orderDate >= start) & (df.orderDate <= end)
    df_full = df[mask]
    te_size = 10000
    tr_size = 10000
    X, Y, fts = dmc.transformation.transform_preserving_header(df_full, scaler=dmc.transformation.scale_features, binary_target=True)
    train = X[:tr_size], Y[:tr_size]
    test = X[tr_size:tr_size+te_size], Y[tr_size:tr_size+te_size]

    res_tree = dmc.evaluation.evaluate_features_leaving_one_out(train[0], train[1], test[0], test[1], fts, dmc.classifiers.DecisionTree)
    for feature in feature_set:
            impact_map[feature] = impact_map[feature] + res_tree.decrement[feature]
            
for key, value in impact_map.items():
    impact_map[key] = value / n

import operator
sorted_impact_list = sorted(impact_map.items(), key=operator.itemgetter(1))
print(sorted_impact_list)

0


/home/bpm1/BigData/run-dmc/venv/lib/python3.4/site-packages/pandas/core/generic.py:2701: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


1
2
3
4
[('t_isWeekend_A', -0.0021200000000000329), ('price', -0.0017400000000000527), ('t_article_priceChangeSTD_A', -0.0017000000000000127), ('totalSavings', -0.0016800000000000369), ('surplusArticleQuantity', -0.0016800000000000369), ('t_order_boughtArticleCount', -0.0016600000000000169), ('t_isGift', -0.0015000000000000568), ('t_article_availableColors', -0.0015000000000000568), ('colorReturnProb', -0.0014000000000000013), ('t_wsv', -0.0012800000000000368), ('t_order_sameArticlesCount', -0.0012800000000000144), ('t_customer_boughtArticleCount', -0.0010600000000000164), ('t_order_boughtArticleTypeCount', -0.001000000000000023), ('orderDay', -0.001000000000000023), ('totalOrderShare', -0.00089999999999998968), ('quantity', -0.00078000000000002514), ('t_isTypePants', -0.00076000000000002736), ('t_order_sameArticlesCount_DiffSize', -0.00064000000000001838), ('t_sizeCodeNumerized', -0.00054000000000000716), ('t_reducedPaymentMethod_A', -0.00044000000000001818), ('sizeCode', -0.000440000